In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matrixprofile import *
import seaborn as sns
import stumpy
from matrixprofile.discords import discords
import ast  # For parsing strings back to lists

In [161]:
def load_npy(filename):
    return np.load(filename)


dir_path = 'cleaned_time_series/'
len_threshold = 1280
X, y, ids = [], [], []

for file in os.listdir(dir_path):
    if os.path.splitext(file)[1] != '.npy':
        continue

    split = file.split("_")
    ids.append(split[0])  # track_id
    y.append(split[1][:-4])  # genre
    ts = load_npy(dir_path + file)

    if len(ts) > len_threshold:
        ts = ts[0:len_threshold]
    else:
        # pad = [np.mean(ts[:-5])] * (len_threshold-len(ts)) # fill by mean value of last n observations
        pad = [ts[-1]] * (len_threshold - len(ts))  # fill with last observation
        ts = np.append(ts, pad)

    X.append([ts])

X, y, ids = np.array(X), np.array(y), np.array(ids)
print(len(X))

10000


**Analysis of the dataset to find motifs and/or anomalies**

# Matrix Profile (Overview)
• The Matrix Profile (MP) is a data structure that annotates a TS and
can be exploited for many purposed: e.g. efficient Motif Discovery.

## Here using matrixprofile library (the one given by the professor)

In [ ]:
plt.plot(X[0,-1].T)
plt.show()

In [ ]:
w = 50
ts = pd.Series(X[0,-1].T)
mp, mpi = matrixProfile.stomp(ts.values, w)
'''
The function *stomp* returns two arrays:
mp (Matrix Profile): An array where each element represents the smallest distance between a subsequence in the time series and its nearest, non-trivial matching subsequence.
mpi (Matrix Profile Index): An array of the indices where these minimum distances occur.
'''
plt.plot(mp)
plt.show()

## Here using stumpy library

In [ ]:
X[0,-1].T

In [ ]:
m = 50
y = X[0,-1].T
x = range(0,1280)
matrix_profile = stumpy.stump(y, m)

In [ ]:
matrix_profile

In [ ]:
matrix_profile_df = pd.DataFrame(matrix_profile, columns=['profile', 'profile index', 'left profile index', 'right profile index'])

In [ ]:
matrix_profile_df

# Motif discovery (Overview)

## Using the matrix profile calculated by stumpy

The best motif is the one where the profile is the smallest (since the profile is the distance value)

Checking for the minimum will give us two matches, each of these should refer to each other which can be see by looking at the profile index:

In [ ]:
best_motif = matrix_profile_df[matrix_profile_df['profile'] == matrix_profile_df['profile'].min()]
best_motif

We can plot this motif:

In [ ]:
from matplotlib.patches import Rectangle

profile_df = matrix_profile_df[['profile']]

fig, ax = plt.subplots(2, figsize=(16,8), sharex=True)
g1 = sns.lineplot(y=y, x=x, ax=ax[0])
g2 = sns.lineplot(data=profile_df, ax=ax[1])

for idx in best_motif.index.to_list():
    g1.axvline(x=idx, color="green")
    g2.axvline(x=idx, color="green")
    rect = Rectangle((idx, 0), m, 40, facecolor="lightgrey")
    g1.add_patch(rect)

We can also see the above zoomed in:

In [ ]:
fig, ax = plt.subplots(figsize=(16,4))

for idx in best_motif.index.to_list():
    plot_y = y.iloc[idx:(idx+m)].to_list()
    sns.lineplot(data=plot_y, ax=ax)

Comparatively, we can compare two random subsequences which don't have any specific relation



In [ ]:
fig, ax = plt.subplots(figsize=(16,4))

for idx in [0, 1000]:
    plot_y = y.iloc[idx:(idx+m)].to_list()
    sns.lineplot(data=plot_y, ax=ax)

## Find a discord/anomaly

Potential **discords/anomalies** can be located as data that's most different to any existing datapoints, this can be found by finding the max profile distance. We can find the anomaly segment by getting this value and plotting it below:

In [ ]:
discord = matrix_profile_df[matrix_profile_df['profile'] == matrix_profile_df['profile'].max()]
discord

In [ ]:
fig, ax = plt.subplots(2, figsize=(16,8), sharex=True)
g1 = sns.lineplot(y=y, x=x, ax=ax[0])
g2 = sns.lineplot(data=profile_df, ax=ax[1])


rect = Rectangle((discord.index[0], 0), m, 40, facecolor="lightgrey")
g1.add_patch(rect)
g2.axvline(x=[discord.index[0]], color='C1')

## Using the matrix profile calculated by matrixprofile library (the one given by the professor)

In [ ]:
mo, mod  = motifs.motifs(ts.values, (mp, mpi), max_motifs=1)

Parameters
- max_motifs: stop finding new motifs once we have max_motifs
- radius: For each motif found, find neighbors that are within radius*motif_mp of the first.
- n_neighbors: number of neighbors from the first to find. If it is None, find all.
- ex_zone: minimum distance between indices for after each subsequence is identified. Defaults to m/2 where m is the subsequence length. If ex_zone = 0, only the found index is exclude, if ex_zone = 1 then if idx is found as a motif idx-1, idx, idx+1 are excluded.

Returns
The function returns a tuple (top_motifs, distances) which are lists of the same length.

- top_motifs: This is a list of the indices found for each motif. The first index is the nth motif followed by all nearest neighbors found sorted by distances.
- distances: Minimum Matrix profile value for each motif set.


In [ ]:
mo

In [ ]:
mod

In [ ]:
plt.plot(ts.values)
colors = ['r', 'g', 'k', 'b', 'y'][:len(mo)]
for m, d, c in zip(mo, mod, colors):
    for i in m:
        m_shape = ts.values[i:i+w]
        plt.plot(range(i,i+w), m_shape, color=c, lw=3)

plt.show()

In [ ]:
for m, d, c in zip(mo, mod, colors):
    for i in m:
        m_shape = ts.values[i:i+w]
        plt.plot(range(i,i+w), m_shape, color=c, lw=3)
    plt.show()

## find a discord/anomaly (using matrixprofile library) (the one given by the professor)

Parameters  
- mp: matrix profile numpy array
- k: the number of discords to discover
- ex_zone: the number of samples to exclude and set to Inf on either side of a found discord    

Returns 
 - a list of indexes represent the discord starting locations. MaxInt indicates there were no more discords that could be found due to too many exclusions or profile being too small. Discord start indices are sorted by highest matrix profile value.

In [ ]:
anoms = discords(mp, ex_zone=3, k=1)

In [ ]:
anoms

In [ ]:
plt.plot(ts.values)
colors = ['r', 'g', 'k', 'b', 'y'][:len(mo)]
for a, c in zip(anoms, colors):
    a_shape = ts.values[a:a+w]
    plt.plot(range(a, a+w), a_shape, color=c, lw=3)

plt.show()

In [ ]:
plt.figure(figsize=(8, 3))
x = ts.values
plt.plot(range(len(x)), x, marker='o', color='r')
plt.xticks(range(len(x)))
plt.grid()
plt.show()

In [ ]:
w = 3
mp = np.array([np.inf] * (len(x) - w + 1))
for i in range(len(x) - w + 1):
    #print('a', x[i:i+w])
    for j in range(len(x) - w + 1):
        if i == j:
            continue
        #print('b', x[j:j+w])
        val = 0
        for k in range(w):
            val += np.abs(x[i + k] - x[j + k])
        #print(val)
        mp[i] = min(mp[i], val)
    #print('')
        

In [ ]:
mp

In [ ]:
plt.figure(figsize=(8, 3))
plt.plot(mp, marker='o', color='b')
plt.xticks(range(len(mp)))
plt.grid()
plt.show()

In [ ]:
mp, mpi = matrixProfile.naiveMP(ts.values, m=12)

# Matrix Profile in entire dataset

## Using stumpy library

## Using matrixprofile library (the one given by the professor)

This code will find the best motif and discord for each time series in the dataset and save them to CSV files.

It's possibile to change the window size, number of motifs and number of discords to find by changing the values of `w`, `n_motifs` and `n_discords` respectively.

In [ ]:
# Initialize the directory paths
best_motif_dir = 'best_motifs'
best_discord_dir = 'best_discord'

# Create directories if they don't exist
os.makedirs(best_motif_dir, exist_ok=True)
os.makedirs(best_discord_dir, exist_ok=True)

w = 40 # window size
n_motifs = 3 # number of motifs to find
n_discords = 2 # number of discords to find

for i in range(len(X)):
    ts = pd.Series(X[i,-1].T)
    mp, mpi = matrixProfile.stomp(ts.values, w)
    mo, mod  = motifs.motifs(ts.values, (mp, mpi), max_motifs=n_motifs)
    anoms = discords(mp, ex_zone=3, k=n_discords)


    motif_df = pd.DataFrame({'Motifs': [str(m) for m in mo]})
    motif_df.to_csv(os.path.join(best_motif_dir, f'best_motif_series_{i}.csv'), index=False)

    print(f"Best motif for {i}: {mo}")
    print(f"Discord for {i}: {anoms}")

    #best_motif.to_csv(f'best_motifs/best_motif_series_{i}.csv', index=False)
    discord_df = pd.DataFrame({'Discords': anoms})
    discord_df.to_csv(os.path.join(best_discord_dir, f'discord_series_{i}.csv'), index=False)

This code will load the best motifs and discords for each time series from the CSV files and combine them into two DataFrames: one for motifs and one for discords.

In [ ]:
# Number of time series
num_series = 3

# Initialize empty DataFrames for motifs and discords
all_motifs = pd.DataFrame()
all_discords = pd.DataFrame()

# Loop through each time series to gather motifs and discords
for i in range(len(X)):
    # Load motifs and discords from CSV
    best_motif = pd.read_csv(f'best_motifs/best_motif_series_{i}.csv')
    discord = pd.read_csv(f'best_discord/discord_series_{i}.csv')

    # Add a column for series identifier
    best_motif['index'] = i
    best_motif['id'] = ids[i]
    discord['index'] = i
    discord['id'] = ids[i]
    
    # Append the motifs and discords to the respective data frames
    all_motifs = pd.concat([all_motifs, best_motif], ignore_index=True)
    all_discords = pd.concat([all_discords, discord], ignore_index=True)

# Reset index of the final data frames
all_motifs.reset_index(drop=True, inplace=True)
all_discords.reset_index(drop=True, inplace=True)

# Now 'all_motifs' and 'all_discords' DataFrames contain all motifs and discords from all series respectively

Now that we have found the best motifs and discords for each time series, we can plot them. First of all we have to load the motifs and discords from the CSV files and then plot them.

In [ ]:
# Number of time series
num_series = 3

colors = ['g', 'k', 'b', 'y']
# Plotting each series' motifs and discords
for i in range(num_series):
    # Load motifs and discords from CSV
    best_motif = pd.read_csv(f'best_motifs/best_motif_series_{i}.csv')
    discord = pd.read_csv(f'best_discord/discord_series_{i}.csv')

    # Assuming you have the original time series data in 'X'
    y = X[i, -1].T
    

    # Create a plot for the time series and its motifs/discords
    plt.figure(figsize=(14, 5))
    plt.title(f"Time Series {i} - Motifs and Discords")
    plt.plot(y, label='Time Series', color='lightgray')

    # Highlight motifs and discords
    color_index = 0
    for index, motif in enumerate(best_motif['Motifs']):
        motif_indices = ast.literal_eval(motif)  # Convert string back to list
        color = colors[color_index % len(colors)]  # Cycle through colors if there are more motifs than colors
        for start_index in motif_indices:
            plt.plot(range(start_index, start_index + w), y[start_index:start_index + w], label=f'Motif {index + 1}', color=color)
        plt.plot([], [], color=color, label=f'Motif {index + 1}', linewidth=3)  # Add legend entry
        color_index += 1


# Highlight discords
    for discord_index in discord['Discords']:
        start_index = int(discord_index)
        plt.plot(range(start_index, start_index + w), y[start_index:start_index + w], label='Discord', color='red')


    # Ensure legends are not repeatedly added
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()


Now we create a new column in the all_motifs dataframe that contains the number of times each motif is duplicated in the dataframe. We then sort the motifs by the number of duplicates and plot the motifs for each series.

In [ ]:
# In the all_motifs dataframe, create a new column 'n_duplicated' that contains the number of times each motif is duplicated in the dataframe
all_motifs['n_duplicated'] = all_motifs.duplicated(subset='Motifs', keep=False).groupby(all_motifs['Motifs']).transform('sum')

# Sort the motifs by the number of duplicates
all_motifs.sort_values('n_duplicated', ascending=False, inplace=True)

all_motifs.sort_values(by=['Motifs','n_duplicated', 'index'], ascending=[True,False, True], inplace=True)

In [ ]:
motif_copy = all_motifs[all_motifs['n_duplicated'] > 1]
len(motif_copy)

In [ ]:
# Number of time series
num_series = 3

motif_copy = all_motifs[all_motifs['n_duplicated'] > 2]
# drop duplicated ids from motif_copy
motif_copy = motif_copy.drop_duplicates(subset='id', keep='first')

colors = ['g', 'k', 'b', 'y']
# Plotting each series' motifs and discords
for series_id in motif_copy['index'].unique():
    # Load motifs and discords from CSV
    best_motif = motif_copy[motif_copy['index'] == series_id]

    # Assuming you have the original time series data in 'X'
    ts = X[series_id, -1].T
    
    print(ids[series_id])
    print(y[series_id])


# Create a plot for the time series and its motifs/discords
    plt.figure(figsize=(14, 5))
    plt.title(f"Time Series {series_id} - Motifs")
    plt.plot(ts, label='Time Series', color='lightgray')

    # Highlight motifs and discords
    color_index = 0
    for index, motif in enumerate(best_motif['Motifs']):
        motif_indices = ast.literal_eval(motif)  # Convert string back to list
        color = colors[color_index % len(colors)]  # Cycle through colors
        for start_index in motif_indices:
            plt.plot(range(start_index, start_index + w), ts[start_index:start_index + w], label=f'Motif {index + 1} {motif_indices}', color=color)
        plt.plot([], [], color=color, label=f'Motif {index + 1}', linewidth=3)  # Add legend entry
        color_index += 1


    # Ensure legends are not repeatedly added
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.show()


# Shapelets

In [162]:
from tslearn.shapelets import ShapeletModel
from tslearn.shapelets import grabocka_params_to_shapelet_size_dict
from tslearn.utils import to_time_series_dataset
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tslearn.shapelets import LearningShapelets, grabocka_params_to_shapelet_size_dict
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.preprocessing import LabelEncoder

Shapelets are subsequences that can be used to represent a class. Matrix profiles make it possibile to identify these shapelets.

In [163]:
# I need to convert the data genres in y to numerical values
le = LabelEncoder()
y = le.fit_transform(y)

In [164]:
# And that's how we can convert them back
le.inverse_transform([y[0]])

array(['opera'], dtype='<U17')

In [230]:
X.shape

(10000, 1, 1280)

In [232]:
# To work with the tslearn library, we need to reshape the data where the first dimension is the number of time series, the second dimension is the number of points in each time series, and the third dimension is the number of dimensions (in this case, 1 since we have univariate time series).
X_reshaped = X.reshape(X.shape[0], X.shape[2], X.shape[1])

X_reshaped.shape

(10000, 1280, 1)

In [233]:
formatted_dataset = to_time_series_dataset(X_reshaped)

In [234]:
formatted_dataset.shape

(10000, 1280, 1)

In [235]:
X_train, X_test, y_train, y_test = train_test_split(formatted_dataset, y, test_size=0.2, random_state=42, stratify=y)

In [236]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8000, 1280, 1), (2000, 1280, 1), (8000,), (2000,))

In [240]:
adam = Adam(learning_rate=0.01)

In [241]:
%%time
# We will extract 1 shapelet and align it with a time series
shapelet_sizes = {40: 1}

# Define the model and fit it using the training data
shp_clf = LearningShapelets(n_shapelets_per_size=shapelet_sizes,
                            weight_regularizer=0.001,
                            optimizer=adam,
                            max_iter=250,
                            verbose=0,
                            scale=False,
                            random_state=42)
shp_clf.fit(X_train[:5], y_train[:5])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tslearn/shapelets/shapelets.py:354: FutureWarning: The default value for 'scale' is set to False in version 0.4 to ensure backward compatibility, but is likely to change in a future version.
  warnings.warn("The default value for 'scale' is set to False "


CPU times: user 7.27 s, sys: 2.61 s, total: 9.88 s
Wall time: 8.16 s


LearningShapelets(max_iter=250, n_shapelets_per_size={40: 1},
                  optimizer=<keras.src.optimizers.adam.Adam object at 0x33062d7d0>,
                  random_state=42, weight_regularizer=0.001)

In [242]:
# We can extract the shapelets from the model
shapelets = shp_clf.shapelets_
shapelets

array([array([[ 0.09760165],
              [ 0.09742235],
              [ 0.10213374],
              [ 0.34835377],
              [ 0.0039369 ],
              [ 0.13306189],
              [ 0.06537931],
              [ 0.00611399],
              [-0.03769759],
              [-0.13045375],
              [-0.30814815],
              [-0.25300485],
              [-0.43169528],
              [-0.62533832],
              [-0.72372705],
              [-0.76944077],
              [-0.74592543],
              [-0.45441568],
              [-0.47513548],
              [-0.31887195],
              [-0.24783155],
              [-0.18667558],
              [-0.16192733],
              [-0.27676496],
              [-0.17637439],
              [ 0.15752938],
              [ 0.0337313 ],
              [-0.15565577],
              [ 0.09166205],
              [ 0.61208141],
              [ 0.59461826],
              [ 0.53897452],
              [ 0.263881  ],
              [ 0.11792707],
              